#  T-cell vaccine design
Design vaccine(s) to elicit a T-cell response by optimising coverage of potential T-cell epitope (PTEs)

Here the term epitope `e` refers to a potential T-cell epitope (PTE) which is a short subsequence of `k` amino acids and also  represented as a node in the epitope graph `G`.

In [ ]:
from Bio import SeqIO
from bin.predict_immunogenicity import Prediction # TODO: Add this to the repo

import igviz as ig
from itertools import product
import json
import matplotlib.pyplot as plt
import mhcflurry
import networkx as nx
import numpy as np
import pandas as pd
import random
from sklearn import preprocessing
import warnings

In [ ]:
# Change
fasta_path = '../data/test/1_nuc_cds_uniq.fa'
hla_path = '../data/test/hla_ref_set.class_i.txt'
k = 9
m = 1 # number of antigens
aligned = False
decycle = True
weights = {
    # 'evo_rate': 0,
    'freq': 1,
    'immunogenicity': 1,
    'mhc_binding': 1,
}

# pre_msa_path = 'hyphy/pre-msa.bf'
# post_msa_path = 'hyphy/post-msa.bf'
# fasta_nt_path = fasta_path + '_nuc.fas'
# fasta_pr_path = fasta_path + '_protein.fas'
# msa_pr_path = fasta_path + '_protein.msa'
# msa_nt_path = fasta_path + '_nuc.msa'
# tree_path = fasta_path + '.treefile'
# fel_path = msa_nt_path + '.FEL.json'
# slac_path = msa_nt_path + '.SLAC.json'

In [ ]:
supported_alleles = !mhcflurry-predict --list-supported-alleles
supported_alleles = list(supported_alleles)

alleles = []
with open(hla_path, 'r') as f:
    for line in f:
        allele = line.strip().split(',')[0]
        if allele in supported_alleles:
            alleles.append(allele)
        else:
            print(f'Allele {allele} not supported by mhcflurry')

In [ ]:
##################
# Calculate scores
##################

def calc_score(kmers_dict, weights, kmer):
    """
    Returns the overall score for a given k-mer
    :param kmers_dict: Dictionary containing all possible k-mers with their corresponding scores
    :param weights: Dictionary containing keys for each scoring criteria and values corresponding to their weight
    :param kmer: String of k-mer
    :returns: Float of overall score for a given k-mer
    """
    return sum([weights[weight] * kmers_dict[kmer][weight] for weight in weights]) / sum(list(weights.values()))


def predict_immunogenicity(peptides):
    """
    Returns the immunogenicity score for a given list of peptides
    :param peptides: List of peptides
    :returns: Dictionary of peptide to immunogenicity score
    """
    # TODO: Add alleles as a parameter
    predictor = Prediction()
    immunogenicity = predictor.main(peptides=peptides)
    df = pd.DataFrame(immunogenicity[1:], columns=immunogenicity[0])
    scaler = preprocessing.MinMaxScaler()
    df['score'] = scaler.fit_transform(df['score'].values.reshape(-1, 1))
    return df[['peptide', 'score']].set_index('peptide').to_dict()['score']


def predict_mhc_binding(peptides, alleles, aligned=False, percentile_threshold=5):
    """
    Returns the MHC binding score for a given list of peptides and alleles
    :param peptides: List of peptides
    :param alleles: List of alleles
    :param aligned: Boolean indicating if the peptides are aligned
    :param percentile_threshold: Integer indicating the percentile threshold for MHC binding
    :returns: Dictionary of peptide to MHC binding score
    """
    !mhcflurry-downloads fetch models_class1_presentation
    predictor = mhcflurry.Class1PresentationPredictor.load()
    if aligned:
        peptides_dict = {e.replace('-',''):e for e in peptides}
        peptides = list(peptides_dict.keys())
    preds = [predictor.predict(peptides, [allele], include_affinity_percentile=True, verbose=0) for allele in alleles]
    preds_df = pd.concat(preds, axis=0)
    if aligned:
        preds_df['peptide'] = preds_df['peptide'].map(peptides_dict)
    preds_df['score'] = preds_df['affinity_percentile'].apply(lambda x: 1 if x < percentile_threshold else 0)
    preds_df = preds_df.groupby('peptide').sum().reset_index() #.median()
    preds_df['score'] = preds_df['score'] / len(alleles)
    # preds_df['score'] = 1 - preds_df['affinity_percentile'] / 100
    return preds_df[['peptide', 'score']].set_index('peptide').to_dict()['score']

# def get_lrt(fel_path):
#     """
#     Returns the likelihood-ratio test (LRT) with sign (+/-) to indicate if the codon is under posotive or negative selection for each codon
#     :param fel_path: String path to HyPhy FEL JSON ouput
#     :returns: List of LRT values for each codon with sign to indicate selection type
#     """
#     fel = json.load(open(fel_path))
#     headers = [header[0] for header in fel['MLE']['headers']] # fel['MLE']['headers']
#     content = fel['MLE']['content']['0']
#     df = pd.DataFrame(content, columns=headers)
#     df['omega'] = df['beta'] / df['alpha']
#     df['LRT_selection'] = preprocessing.normalize([np.array(df['LRT'])])[0]
#     df['LRT_selection'] = df.apply(lambda x: x['LRT_selection'] if x['alpha'] < x['beta'] else x['LRT_selection']*-1, axis=1)
#     return df['LRT_selection'].tolist()


def get_evo_rate(slac_path):
    """
    Returns the evolution rate score which is the the absolute number of synonymous (-1) + nonsynonymous (1) sites at each position in the alignment
    :param slac_path: String path to HyPhy SLAC JSON ouput
    :returns: List of evolution rate score for each codon
    """
    slac = json.load(open(slac_path))
    headers = [header[0] for header in slac['MLE']['headers']]
    content = slac['MLE']['content']['0']['by-site']['RESOLVED'] #['AVERAGED']
    df = pd.DataFrame(content, columns=headers)[['S', 'N']]
    # This is a bit counter-intuitive but we set the number of non-synonymous sites (N) to negative even though they're under positive selection
    # This is because we want to select for sites that are under negative (aka purifying) selection
    df['N'] = df['N']*-1
    df['evo_rate'] = df['S'] + df['N']
    scaler = preprocessing.MinMaxScaler()
    df['evo_rate'] = scaler.fit_transform(df['evo_rate'].values.reshape(-1, 1))
    return df['evo_rate'].tolist()


##########################################
# Utils to convert the format of sequences
##########################################

def seq_to_kmers(seq, k):
    """
    Returns a list of k-mers of length k for a given string of amino acid sequence
    """
    return [seq[i:i+k] for i in range(len(seq) - k + 1)]


# For aligned sequences
def seqs_dict_to_df(seqs_dict, evo_rate):
    """
    Returns a dataframe of sequences, their names and corresponding evolutionary rate for each codon
    :param seqs_dict: Dictionary where the keys are sequence IDs and the values are aligned amino acid sequence strings
    :param evo_rate: List containing the evolutionary rate for each codon/site in the MSA 
    :returns: Dataframe of sequences, their names and corresponding evolutionary rate for each codon
    """
    seqs = [list(seq) for seq in list(seqs_dict.values())]
    seqs.insert(0, evo_rate)
    names = ['Evo_Rate'] + list(seqs_dict.keys())
    df = pd.DataFrame(seqs)
    df.index = names
    return df


def seqs_df_to_kmers_dict(df, weights, k=9):
    """
    Returns a dictionary of all possible k-mers and their evolutionary rate, position, frequency and overall score
    :param df: Dataframe of amino acid sequences, their names and corresponding evolutionary rate for each codon
    :param k: Integer for substring length
    :param weights: Dictionary containing keys for each scoring criteria and values corresponding to their weight
    :returns: Dictionary containing all possible k-mers with their evolutionary rate, position, frequency and overall score
    """
    kmers_dict = {}
    for i in range(len(df.columns) - k + 1):
        kmers = df.loc[:, i:i+k-1]
        evo_rate = kmers.loc['Evo_Rate'].tolist()
        evo_rate = sum(evo_rate) / len(evo_rate)
        pos = i+1
        for index, row in kmers.iterrows():
            if index != 'Evo_Rate':
                kmer = ''.join(row.tolist())
                if kmer not in kmers_dict:
                    kmers_dict[kmer] = {'evo_rate':evo_rate, 'pos':pos, 'freq':1}
                else:
                    if pos == kmers_dict[kmer]['pos']:
                        kmers_dict[kmer]['freq'] += 1
                    else:
                        warnings.warn(f'k-mer {kmer} found at multiple positions {pos},{kmers_dict[kmer]["pos"]}. This was not expected to happen and may cause issues.')
    for kmer in kmers_dict:
        # Convert sequence counts to frequency
        kmers_dict[kmer]['freq'] = kmers_dict[kmer]['freq'] / len(df.columns)
        # Calculate score using weights
        kmers_dict[kmer]['score'] = calc_score(kmers_dict, weights, kmer)
    return kmers_dict


# For unaligned sequences
def seqs_to_kmers_dict(seqs, k=9, weights=weights, alleles=None, aligned=False):
    """
    Returns a dictionary of all possible k-mers and their frequencies for a given list of sequences and value of k
    :param seqs: List of amino acid sequences
    :param k: Integer for substring length 
    :param alleles: List of MHC alleles to use for MHC binding prediction
    :param aligned: Boolean indicating whether the sequences are aligned or unaligned
    :returns: Dictionary containing all possible k-mers and their frequencies
    """
    kmers_dict = {}
    N = len(seqs)
    for seq in seqs:
        # Get a unique set of k-mers for each sequence
        kmers = set(seq_to_kmers(seq, k))
        # Count the number of seqs for each k-mer (n)
        for e in kmers:
            if e in kmers_dict:
                kmers_dict[e]['freq'] += 1
            else:
                kmers_dict[e] = {'freq': 1, 'pos': 0, 'evo_rate': 0, 'immunogenicity':0, 'mhc_binding': 0, 'score': 0}

    immunogenicity_dict = {e:0 for e in kmers_dict}
    mhc_binding_dict = {e:0 for e in kmers_dict}
    peptides = list(kmers_dict.keys())

    if weights['immunogenicity']:
        immunogenicity_dict = predict_immunogenicity(peptides)
        
    if weights['mhc_binding']:
        mhc_binding_dict = predict_mhc_binding(peptides, alleles, aligned)

    for e, d in kmers_dict.items():
        kmers_dict[e]['freq'] = d['freq']/N
        kmers_dict[e]['immunogenicity'] = immunogenicity_dict[e]
        kmers_dict[e]['mhc_binding'] = mhc_binding_dict[e]
        kmers_dict[e]['score'] = calc_score(kmers_dict, weights, e)

    return kmers_dict


def path_to_seq(path):
    """
    Returns an AA string for a list of epitopes (path)
    """
    seq = [path[0]] + [e[-1] for e in path[1:]]
    return ''.join(seq)


############################################
# Utils to get the min/max index from a list
############################################

def argmax(lst):
    """
    Returns the index for the maximum value in a list
    """
    return lst.index(max(lst))


def argmin(lst):
    """
    Returns the index for the minimum value in a list
    """
    return lst.index(min(lst))


#####################################
# Utils to retrieve info from a graph
#####################################

def P(G, e):
    """
    Returns the predecessors for a given graph G and node e
    :param G: Directed Graph containing epitopes
    :param e: String for a given potential T-cell epitope (PTE)
    :returns: List of predecessors
    """
    return list(G.predecessors(e))


def S(G, e):
    """
    Returns the successors for a given graph G and node e
    :param G: Directed Graph containing epitopes
    :param e: String for a given potential T-cell epitope (PTE)
    :returns: List of successors
    """
    return list(G.successors(e))


def f(G, e, f='Score'):
    """
    Returns the feature for a given epitope e eg frequency or score in the population
    :param G: Directed Graph containing epitopes
    :param e: String for a given potential T-cell epitope (PTE)
    :param e: String for the node feature (default = 'Score')
    :returns: Float for the epitope feature eg score
    """
    return G.nodes[e][f]


############################################
# Decycling - remove all cycles from a graph
############################################

def decycle_graph(G):
    """
    Return a Directed Graph with no cycles
    :param G: Directed Graph containing epitopes
    :returns: Directed Graph containing epitopes and no cycles
    """
    # j is a list of all compnents; each component is a list of nodes in G
    components = list(nx.strongly_connected_components(G))
    # Discard all single node components - no cycles there!
    components = [j for j in components if len(j) != 1]
    if len(components) != 0:
        for j in components:
            # Randomly choose two nodes from the selected component
            ea, eb = random.sample(list(j), k=2)
            cycle = cycle_from_two_nodes(G, ea, eb)
            if cycle:
                ea, eb = weak_edge_in_cycle(G, cycle)
                G.remove_edge(ea, eb)
                # Repeat until graph is acyclic
                G = decycle_graph(G)
    return G


def cycle_from_two_nodes(G, ea, eb):
    """
    Returns the cycle (i.e. path that starts and ends in with the same epitope) for two nodes
    :param G: Directed Graph containing epitopes
    :param ea: String for the first given potential T-cell epitope (PTE)
    :param eb: String for the second given potential T-cell epitope (PTE)
    :returns: List of epitope strings on path that is a cycle
    """
    try:
        path_ab = nx.shortest_path(G, source=ea, target=eb)
        path_ba = nx.shortest_path(G, source=eb, target=ea)
        # Merge two paths into a cycle
        cycle = path_ab[:-1] + path_ba
    except nx.NetworkXNoPath:
        cycle = []
    return cycle


def weak_edge_in_cycle(G, cycle):
    """
    Returns the weak edge (edge with the lowest score) in a cycle
    :param G: Directed Graph containing epitopes
    :param cycle: List of epitope strings on path that is a cycle
    :returns: Tuple for the weak edge containing the two epitope strings
    """
    edges = seq_to_kmers(cycle, k=2)
    values = []
    for ea, eb in edges:
        # v is heuristic “value” of edge
        v = f(G, ea) + f(G, eb)
        # Add value if cutting edge would isolate ea
        if len(S(G, ea)) == 1:
            v = v + f(G, ea)
        # Add value if cutting edge would isolate eb
        if len(P(G, eb)) == 1:
            v = v + f(G, eb)
        values.append(v)
    ea, eb = edges[argmin(values)]
    return ea, eb


###############################################
# Find optimal path through a graph of epitopes
###############################################

def find_optimal_path(G):
    """
    Returns the optimal path through a graph of epitopes
    :param G: Directed Graph containing epitopes
    :returns: List of epitope strings on the optimal path
    """
    # Forward loop - compute F(e)
    for e in G.nodes:
        F(G, e)
    # Backward loop - build the path that achieves the maximal score
    path = backward(G)
    return path


def F(G, e):
    """
    Returns the maximum total score over all paths that end in e
    :param G: Directed Graph containing epitopes
    :param e: String for a given potential T-cell epitope (PTE)
    :returns: Float for the maximum total epitope score
    """
    # Use precomputed F(e) if it already exists for the epitope
    if 'F(e)' not in G.nodes[e]:
        predecessors = P(G, e)
        if not predecessors:
            # If the set of predecessors P(e) is empty, then F(e) = f(e)
            Fe = f(G, e)
        else:
            # If the set of predecessors P(e) is not empty, then F(e) = f(e) + max(F(P(e)))
            Fe = f(G, e) + max([F(G, pe) for pe in predecessors])
        # Save F(e) to the graph for this epitope
        nx.set_node_attributes(G, {e: Fe}, 'F(e)')
    return f(G, e, f='F(e)')


def backward(G, path=[]):
    """
    Returns the path that achieves the maximal score
    :param G: Directed Graph containing epitopes
    :param path: List of epitope strings to complete (deafult=[])
    :returns: List of epitope strings on path that achieve maximum score
    """
    # Get the precomputed F(e) from the graph for all epitopes
    Fe_dict = nx.get_node_attributes(G, 'F(e)')
    if not path:
        # Get the epitope with the maximum F(e) as the final epitope in our optimal path
        end_nodes = {e: Fe_dict[e] for e in P(G, 'END')}
        path = [max(end_nodes, key=end_nodes.get)]
    # Get the most recently added epitope e and it's predecessors P(e)
    e = path[0]
    predecessors = P(G, e)
    if predecessors[0] != 'BEGIN':
        # Add the best (highest F(e)) predecessor P(e) of epitope e to our path
        i = argmax([Fe_dict[pe] for pe in predecessors])
        path.insert(0, predecessors[i])
        # Repeat until you get to the start
        backward(G, path)
    return path


###########################################################
# Cocktail: Find (and iteratively refine) a set of antigens
###########################################################

def cocktail(G, m, refine=True, score='Score'):
    """
    Returns a list of m antigens
    :param G: Directed Graph containing epitopes
    :param m: Integer for number of antigens
    :param refine: Boolean for if the antigens should be iteratively refined
    :returns: List containing m antigens
    """
    Q = [] # vaccine
    # Save original epitope score so it can be reset later
    score_dict = nx.get_node_attributes(G, score)
    for n in range(0, m):
        # Compute and save next antigen sequence
        q = find_optimal_path(G)
        # Add q to vaccine
        Q.append(q)
        # No credit for including e in subsequent antigens
        for e in q:
            nx.set_node_attributes(G, {e: 0}, score)
        # Remove F(e) so it's recomputed using the updated scores
        for (e,d) in G.nodes(data=True):
            del d['F(e)']
    # Reset to the original scores
    nx.set_node_attributes(G, score_dict, score)
    # Optional: Repeat - iterative refinement
    if refine:
        Q = iterative_refinement(G, Q)     
    return Q


def iterative_refinement(G, Q, score='Score'):
    """
    Returns a list of iteratively refined antigens
    :param G: Directed Graph containing epitopes
    :param Q: List containing antigens
    :returns: List containing iteratively refined antigens
    """
    m = len(Q)
    # Save original epitope score so it can be reset later
    score_dict = nx.get_node_attributes(G, score)
    while True:
        for n in range(0, m):
            prev_Q = Q
            # Remove sequence q from vaccine Q
            q = Q[n]
            Q.remove(q)
            # No credit for including e in existing antigens
            for q in Q:
                for e in q:
                    nx.set_node_attributes(G, {e: 0}, score)
            # Compute replacement for old sequence q
            q = find_optimal_path(G)
            # Add q to vaccine
            Q.insert(n, q)
            # Reset to the original scores
            nx.set_node_attributes(G, score_dict, score)
            # Remove F(e) so it's recomputed using the updated scores
            for (e,d) in G.nodes(data=True):
                del d['F(e)']
            if Q == prev_Q:
                return Q


##################################################
# Construct/visualise the epitope graph and scores
##################################################

def construct_graph(kmers_dict, decycle=True, aligned=False, edge_colour='#BFBFBF'):
    """
    Return a Directed Graph with unique k-mers as nodes, where overlapping k-mers are connected by edges
    :param kmers_dict: Dictionary containing k-mers and their counts
    :param decycle: Boolean if the output graph should be decycled (default=True)
    :param edge_colour: String for edge colour (default='black')
    :returns: Directed Graph containing epitopes
    """
    # Create graph
    G = nx.DiGraph()
    # Add nodes - for each unique k-mer
    for e, d in kmers_dict.items():
        freq, immunogenicity, mhc_binding, evo_rate, pos, score = d['freq'], d['immunogenicity'], d['mhc_binding'], d['evo_rate'], d['pos'], d['score']
        G.add_node(e, Frequency=freq, Immunogenicity=immunogenicity, MHC_Binding=mhc_binding, Evo_Rate=evo_rate, pos=(pos, score), Score=score)
    # Add edges - where the last k−1 characters of ea match the first k−1 characters of eb
    for ea, eb in product(G.nodes(), G.nodes()):
        if not G.has_edge(ea, eb) and ea[1:] == eb[:-1]:
            G.add_edge(ea, eb, colour=edge_colour)
    # Decycle graph
    if decycle:
        G = decycle_graph(G)
    # Add begin and end nodes
    begin_nodes = [e for e in list(G.nodes) if not P(G, e)]
    end_nodes = [e for e in list(G.nodes) if not S(G, e)]
    G.add_node('BEGIN', Frequency=0, Immunogenicity=0, MHC_Binding=0, Evo_Rate=0, pos=(0,0), Score=0)
    G.add_node('END', Frequency=0, Immunogenicity=0, MHC_Binding=0, Evo_Rate=0, pos=(0,0), Score=0)
    for e in begin_nodes:
        G.add_edge('BEGIN', e, colour=edge_colour)
    for e in end_nodes:
        G.add_edge(e, 'END', colour=edge_colour)
    # Add the position node attribute (for unaligned sequences)
    if not aligned:
        for e in G.nodes():
            if e != 'BEGIN' and e != 'END':
                pos = nx.shortest_path_length(G, source='BEGIN', target=e)
                score = f(G, e)
                nx.set_node_attributes(G, {e: (pos, score)}, 'pos')
    # Ensure the 'END' node is always at the end of the graph
    end_pos = max([pos[0] for pos in list(nx.get_node_attributes(G, 'pos').values())]) + 1
    nx.set_node_attributes(G, {'END': (end_pos, 0)}, 'pos')
    return G


def plot_graph(G, paths=None, node_size=150, with_labels=False, ylim=([0,1]), interactive=False, colour_by='Score'):
    """
    Plot the epitope graph
    :param G: Directed Graph containing epitopes
    :param paths: List of lists of epitope strings (deafult=None)
    :param node_size: Integer for size of nodes in non-interactive plot (default=150)
    :param with_labels: Boolean for if epitope labels should be displayed in the non-interactive plot (default=False)
    :param ylim: List for y-axis limits (default=[0,1])
    :param interactive: Boolean for if the plot should be interactive (default=False)
    :returns: None
    """
    if interactive:
        fig = ig.plot(G, color_method='Score', node_text=['Score', 'Frequency', 'Immunogenicity', 'MHC_Binding', 'pos']) # layout='spectral','spiral','spring'
        return fig.show()
    else:
        # Define vars
        node_color = list(nx.get_node_attributes(G,colour_by).values())
        pos = nx.get_node_attributes(G, 'pos')
        if paths:
            for path in paths:
                path = ['BEGIN'] + path + ['END']
                for i in range(0,len(path)-1):
                    G.edges[path[i], path[i+1]]['colour'] = 'red'
        edge_colours = nx.get_edge_attributes(G, 'colour')
        # Plot
        fig, ax = plt.subplots(1,figsize=(16,8))
        nx.draw(G, node_color=node_color, pos=pos, node_size=node_size, with_labels=with_labels, edge_color=edge_colours.values(), width=2, font_color='white', ax=ax)
        limits=plt.axis('on')
        max_pos = max([p[0] for p in pos.values()]) + 0.5
        ax.set_xlim([-0.5, max_pos])
        ax.set_ylim(ylim)
        ax.spines.right.set_visible(False)
        ax.spines.top.set_visible(False)
        ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
        ax.tick_params(axis='both', which='major', labelsize=14)
        plt.ylabel('Epitope Score, f(e)', fontsize=18)


def plot_score(G, score='Frequency'):
    """
    Plot the distribution of a given score for the nodes in the graph.
    :param G: NetworkX graph
    :param score: String for the score to be plotted (default='Frequency')
    :returns: Figure
    """
    scores = list(nx.get_node_attributes(G,score).values())
    # Plot
    fig, ax = plt.subplots(1,figsize=(16,8))
    ax.hist(scores, bins=100)
    ax.set_xlabel(score, fontsize=18)
    ax.set_ylabel('Number of PTEs', fontsize=18)
    ax.tick_params(axis='both', which='major', labelsize=14)
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)
    ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
    return fig

## Simple example 

In [ ]:
# Define the input epitopes
# Cyclic example: kmers_dict = {'MSA': 0.6, 'SAM': 0.2, 'AMS': 0.4}
kmers_dict = {
    'MSA': {'evo_rate':0, 'immunogenicity':0, 'mhc_binding':0, 'pos':1, 'freq':0, 'score':0.6},
    'SAM': {'evo_rate':0, 'immunogenicity':0, 'mhc_binding':0, 'pos':2, 'freq':0, 'score':0.2},
    'AMQ': {'evo_rate':0, 'immunogenicity':0, 'mhc_binding':0, 'pos':3, 'freq':0, 'score':0.2},
    'MQL': {'evo_rate':0, 'immunogenicity':0, 'mhc_binding':0, 'pos':4, 'freq':0, 'score':0.2},
    'SAR': {'evo_rate':0, 'immunogenicity':0, 'mhc_binding':0, 'pos':2, 'freq':0, 'score':0.4},
    'MGA': {'evo_rate':0, 'immunogenicity':0, 'mhc_binding':0, 'pos':1, 'freq':0, 'score':0.3},
    'GAR': {'evo_rate':0, 'immunogenicity':0, 'mhc_binding':0, 'pos':2, 'freq':0, 'score':0.7},
    'ARQ': {'evo_rate':0, 'immunogenicity':0, 'mhc_binding':0, 'pos':3, 'freq':0, 'score':0.4},
    'RQL': {'evo_rate':0, 'immunogenicity':0, 'mhc_binding':0, 'pos':4, 'freq':0, 'score':0.4},
}

# Construct the graph
G = construct_graph(kmers_dict, aligned=True)

# Find the optimal path(s) through the graph of epitopes
Q = cocktail(G, m)
print([path_to_seq(path) for path in Q])

# Plot the results
plot_graph(G, paths=Q, node_size=2000, ylim=[-0.1, 1], with_labels=True, interactive=False)

## Perform a Multiple Sequence Alignment (MSA)
The input nucleotide sequences are preprocessed by performing a MSA. This is required to calculate omega (dN/dS) and is done in the following steps:
1. **Pre-MSA:** Run these sequences through `pre-msa.bf` in order to correct frame-shift mutations and translate the resulting sequences to proteins
2. **MSA:** Take the output of step 1 and run in through MAFFT to generate a **protein** MSA
3. **Post-MSA:** Run the protein MSA (from step 2) and the frameshift corrected nucleotide sequences (from step 1) through `post-msa.bf` to obtain a nucleotide MSA. This step will also, compress all identical sequences, i.e. replace them with a single representative sequence.

In [ ]:
# Step 1) Pre-MSA
# !hyphy {pre_msa_path} --input {fasta_path}

In [ ]:
# Step 2) MSA
# !mafft --auto {fasta_pr_path} > {msa_pr_path}

In [ ]:
# Step 3) Post-MSA
# !hyphy {post_msa_path} --protein-msa {msa_pr_path} --nucleotide-sequences {fasta_nt_path} --output {msa_nt_path}

## Compare nonsynoymous (dN) and synonymous (dS) substitution rates

Use HyPhy FEL to infer the number of nonsynoymous (N) and synonymous (S) substitutions on a per-site basis for a given coding alignment and corresponding phylogeny. Report N and S, and combine them to ascertain if N is greater than S:
1. **Tree consutrction:** Consutruct a phylogenetic tree in NEWICK format using IQ-TREE
2. **Compare dN and dS:** using the nucleotide MSA (from MSA step 3) and the phylogenetic tree (from step 1) as input to HyPhy FEL infer the nonsynoymous (dN), synonymous (dS) and estimate the number of synonymous/nonsynonymous sites for each codon/site in the MSA
3. **Get Evolutionary rate:** load the JSON output from HyPhy SLAC (step 2) and adjust the sign (+/-) of the number of synonymous (-1) vs nonsynonymous (1) sites to indicate if the codon is under posotive/negative selection

Note:
- Diversifying or positive selection is where dS/α < dN/β
- Purifying or negative selection is where dS/α > dN/β

We want to select for codons under purifying selection because they're more conserved

In [ ]:
# Step 1) Tree construction
# !iqtree -s {msa_nt_path} -pre {fasta_path} --redo --fast

In [ ]:
# Step 2) Compare dN and dS
# !hyphy slac --alignment {msa_nt_path} --tree {tree_path} --output {slac_path}

In [ ]:
# Step 3) Get Evolutionary Rate
evo_rate = get_evo_rate(slac_path)

## Load the FASTA sequences

In [ ]:
if aligned:
    fasta_path = msa_pr_path
else:
    fasta_path = fasta_pr_path

fasta_seqs = SeqIO.parse(open(fasta_path),'fasta')
seqs_dict = {seq.id:str(seq.seq) for seq in fasta_seqs}

## Split into k-mers
Compute all possible k-mers of length `k` for the given target sequences and for each k-mer record the mean evolutionary rate, position in the alignment, epitope frequency and overall score

In [ ]:
# TODO: Add logic to use the input weights to check if all required inputs are present and only calculate desired scores
if aligned:
    df = seqs_dict_to_df(seqs_dict, evo_rate)
    kmers_dict = seqs_df_to_kmers_dict(df, weights, k)
else:
    kmers_dict = seqs_to_kmers_dict(list(seqs_dict.values()), k=k, weights=weights, alleles=alleles, aligned=aligned)

## Construct the epitope graph
Create a Directed Graph (`DiGraph`) using the `networkx` package, where each epitope `e` is a node and edges connect nodes where the last `k−1` characters of `ea` match the first `k−1` characters of `eb`. For computational convenience, two extra nodes `BEGIN` and `END` are added. The `BEGIN` node connects to all the nodes that lack predecessors (`P(e)`) (corresponding to epitopes that are the first `k` characters in a sequence). Nodes that lack successors (`S(e)`) (because they are the last `k` characters in a sequence) are connected to the `END`. For plotting convenience, the length shortest path to the `BEGIN` node is added as a node attribute

In [ ]:
G = construct_graph(kmers_dict, decycle=decycle, aligned=aligned)

## Assembly 
Take a path through the graph to optimise epitope frequency.

The forward loop computes the function `F(e)` (the largest sum achievable for any path that terminates with the epitope `e`) for all the nodes in a stepwise manner. The backward loop chooses the node with maximum value as the final epitope in our optimal string and works backwards to build the path that achieves the maximal score

In [ ]:
# TODO: Add additional measures to scoring function eg binding affinity prediction

In [ ]:
# Find the optimal path(s) through the graph of epitopes
Q = cocktail(G, m)
[path_to_seq(path) for path in Q]

## Plot the epitope graph
The nodes are the epitopes `e` and the edges connect epitopes whose sequences overlap by `k − 1` amino acids. The x-axis shows the shortest path length to the `BEGIN` node, the y-axis indicates the epitope frequency `f(e)` in this target sequence set. The optimal path is shown in red which corresponds to the protein sequence that maximizes epitope coverage of the population

In [ ]:
# F(e)
plot_graph(G, paths=Q, node_size=25, with_labels=False, ylim=[0, 1], interactive=False, colour_by='Score')

## Evaluate the implemented scores

In [ ]:
# Get the scores of each node in the path
pos = [G.nodes[node]['pos'][0] for node in Q[0]]
freq = [G.nodes[node]['Frequency'] for node in Q[0]]
mhc_binding = [G.nodes[node]['MHC_Binding'] for node in Q[0]]
immunogenicity = [G.nodes[node]['Immunogenicity'] for node in Q[0]]

# Plot stacked area
fig, ax = plt.subplots(1,figsize=(16,8))
ax.stackplot(pos, freq, mhc_binding, immunogenicity, labels=['Frequency', 'MHC Binding', 'Immunogenicity'])
ax.set_xlabel('Position', fontsize=18)
ax.set_ylabel('Score', fontsize=18)
ax.tick_params(axis='both', which='major', labelsize=14)
ax.spines.right.set_visible(False)
ax.spines.top.set_visible(False)
ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
ax.legend(loc='upper left', fontsize=14)
ax.set_xlim([0, max(pos)])
plt.show()


View the distribution of the scores for all the epitopes in the graph

In [ ]:
hist_plot = plot_score(G, score='Frequency')
# hist_plot = plot_score(G, score='MHC_Binding')
# hist_plot = plot_score(G, score='Immunogenicity')
hist_plot.show()

Compare the values for different scores to see if each score is providing additional information

In [ ]:
freq_dict = nx.get_node_attributes(G, 'Frequency')
freqs = list(freq_dict.values())

immunogenicity_dict = nx.get_node_attributes(G, 'Immunogenicity')
immunogenecities = list(immunogenicity_dict.values())

mhc_binding_dict = nx.get_node_attributes(G, 'MHC_Binding')
mhc_bindings = list(mhc_binding_dict.values())

plt.scatter(x=immunogenecities,y=mhc_bindings)
plt.xlabel('immunogenecities')
plt.ylabel('MHC_Binding')
plt.show()